## 1. Load dataset

The example datasets contain molecule structure (SMILES) and measured bioactivity (pKi or IC50) – the higher the better. Each SMILES is converted to a Mol object in RDKit.

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem

In [2]:
def reg_to_clf(y):
    return np.where(np.array(y) > 6, 1, 0)

In [3]:
data_train = pd.read_csv('data/CHEMBL1824/train.csv', header=None)
data_test = pd.read_csv('data/CHEMBL1824/test.csv', header=None)

In [4]:
smi_train, prop_train = data_train[0].to_list(), reg_to_clf(data_train[1].to_list())
smi_test, prop_test = data_test[0].to_list(), reg_to_clf(data_test[1].to_list())

In [5]:
mols_train, y_train = [], []
for smi, prop in zip(smi_train, prop_train):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        mols_train.append(mol)
        y_train.append(prop)

In [6]:
mols_test, y_test = [], []
for smi, prop in zip(smi_test, prop_test):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        mols_test.append(mol)
        y_test.append(prop)

## 1.5 Reduce the dataset size for faster pipeline (for playing around)

In [7]:
# mols_train, y_train = mols_train[:80], y_train[:80]
# mols_test, y_test = mols_test[:20], y_test[:20]

## 2. Conformer generation

For each molecule, an ensemble of conformers is generated. Then, molecules for which conformer generation failed are filtered out from both, the training and test set. Generated conformers can be accessed by mol.GetConformers(confID=0).

In [8]:
from qsarmil.conformer import RDKitConformerGenerator

from qsarmil.utils.logging import FailedConformer, FailedDescriptor

In [9]:
conf_gen = RDKitConformerGenerator(num_conf=10, num_cpu=40)

In [10]:
confs_train = conf_gen.run(mols_train)

tmp = [(c, y) for c, y in zip(confs_train, y_train) if not isinstance(c, FailedConformer)]
confs_train, y_train = zip(*tmp) 
confs_train, y_train = list(confs_train), list(y_train)

Generating conformers: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1667/1667 [01:10<00:00, 23.69it/s]


In [11]:
confs_test = conf_gen.run(mols_test)

tmp = [(c, y) for c, y in zip(confs_test, y_test) if not isinstance(c, FailedConformer)]
confs_test, y_test = zip(*tmp) 
confs_test, y_test = list(confs_test), list(y_test)

Generating conformers: 100%|███████████████████████████████████████████████████████████████████████████████████████| 556/556 [00:28<00:00, 19.71it/s]


## 3. Descriptor calculation

Then, for each molecule with associated conformers 3D descriptors are calculated. Here, a descriptor wrapper is used, which is designed to apply descriptor calculators from external packages. The resulting descriptors are a list of 2D arrays (bags). Also, the resulting descriptors are scaled.

In [12]:
from qsarmil.descriptor.rdkit import (RDKitGEOM, 
                                      RDKitAUTOCORR, 
                                      RDKitRDF, 
                                      RDKitMORSE, 
                                      RDKitWHIM, 
                                      RDKitGETAWAY)

from molfeat.calc import Pharmacophore3D, USRDescriptors, ElectroShapeDescriptors

from qsarmil.descriptor.wrapper import DescriptorWrapper

from qsarmil.mil.preprocessing import BagMinMaxScaler

In [13]:
desc_calc = DescriptorWrapper(RDKitRDF())

In [14]:
x_train = desc_calc.transform(confs_train)
x_test = desc_calc.transform(confs_test)

In [15]:
scaler = BagMinMaxScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

## 4. Model training

In [16]:
from sklearn.metrics import r2_score, accuracy_score

from sklearn.ensemble import RandomForestClassifier

from qsarmil.mil.wrapper import InstanceWrapper, BagWrapper

from qsarmil.mil.network.regressor import InstanceNetworkRegressor, BagNetworkRegressor
from qsarmil.mil.network.classifier import InstanceNetworkClassifier, BagNetworkClassifier

from qsarmil.mil.network.regressor import (AttentionNetworkRegressor, 
                                           SelfAttentionNetworkRegressor,
                                           GatedAttentionNetworkRegressor,
                                           TemperatureAttentionNetworkRegressor,
                                           GaussianPoolingNetworkRegressor,
                                           DynamicPoolingNetworkRegressor)

from qsarmil.mil.network.classifier import (AttentionNetworkClassifier, 
                                            SelfAttentionNetworkClassifier,
                                            GatedAttentionNetworkClassifier,
                                            TemperatureAttentionNetworkClassifier,
                                            GaussianPoolingNetworkClassifier,
                                            DynamicPoolingNetworkClassifier)

In [17]:
network_hparams = {'hidden_layer_sizes':(256, 128, 64),
                   'num_epoch':300,
                   'batch_size':128,
                   'learning_rate':0.001,
                   'weight_decay':0.001,
                   'instance_weight_dropout':0.01,
                   'init_cuda':False,
                   'verbose':False}

In [18]:
method_list = [
               ("MeanInstanceWrapper", InstanceWrapper(estimator=RandomForestClassifier(), pool="mean")), 
               ("MaxInstanceWrapper", InstanceWrapper(RandomForestClassifier(), pool="max")), 
               ("MeanBagWrapper", BagWrapper(RandomForestClassifier(), pool="mean")), 
               ("MaxBagWrapper", BagWrapper(RandomForestClassifier(), pool="max")), 
               ("MinBagWrapper", BagWrapper(RandomForestClassifier(), pool="min")), 
               ("ExtremeBagWrapper", BagWrapper(RandomForestClassifier(), pool="extreme")),
               ("MeanInstanceNetwork", InstanceNetworkClassifier(**network_hparams, pool="mean")),
               ("MaxInstanceNetwork", InstanceNetworkClassifier(**network_hparams, pool="max")),
               ("MeanBagNetwork", BagNetworkClassifier(**network_hparams, pool="mean")),
               ("MaxBagNetwork", BagNetworkClassifier(**network_hparams, pool="max")),
               ("AttentionNetworkClassifier", AttentionNetworkClassifier(**network_hparams)),
               ("SelfAttentionNetworkClassifier", SelfAttentionNetworkClassifier(**network_hparams)),
               ("GatedAttentionNetworkClassifier", GatedAttentionNetworkClassifier(**network_hparams)),
               ("TemperatureAttentionNetworkClassifier", TemperatureAttentionNetworkClassifier(**network_hparams)),
               ("GaussianPoolingNetworkClassifier", GaussianPoolingNetworkClassifier(**network_hparams)),
               ("DynamicPoolingNetworkClassifier", DynamicPoolingNetworkClassifier(**network_hparams))
              ]

In [19]:
res_df = pd.DataFrame()
for method_name, model in method_list:
    model.fit(x_train_scaled, y_train)
    
    y_prob = model.predict(x_test_scaled)
    y_pred = np.where(y_prob > 0.5, 1, 0)
    
    res_df.loc[method_name, "ACC"] = accuracy_score(y_test, y_pred)

In [20]:
res_df.sort_values(by="ACC", ascending=False)

,ACC
MeanInstanceNetwork,0.897482
TemperatureAttentionNetworkClassifier,0.890288
GatedAttentionNetworkClassifier,0.881295
GaussianPoolingNetworkClassifier,0.879496
MeanBagNetwork,0.875899
SelfAttentionNetworkClassifier,0.874101
AttentionNetworkClassifier,0.868705
DynamicPoolingNetworkClassifier,0.857914
MeanInstanceWrapper,0.856115
MaxBagNetwork,0.852518


## 5. Key Instance Detection

Some MIL algorithms can identify key instances (if they have get_instance_weights method). In this section, AttentionNetworkRegressor is used to estimate the conformer weights. Here, different 3D descriptors are used to estimate the weight distribution depending on the representation type.

**Conclusion:** With current representations available the weight distribution is not definitive (almost uniform).

In [21]:
from qsarmil.descriptor.rdkit import (RDKitGEOM, 
                                      RDKitAUTOCORR, 
                                      RDKitRDF, 
                                      RDKitMORSE, 
                                      RDKitWHIM, 
                                      RDKitGETAWAY)

from molfeat.calc import (Pharmacophore3D, 
                          USRDescriptors, 
                          ElectroShapeDescriptors)

from qsarmil.descriptor.wrapper import DescriptorWrapper

In [22]:
desc_list = [
             ("RDKitGEOM", DescriptorWrapper(RDKitGEOM())),
             ("RDKitAUTOCORR", DescriptorWrapper(RDKitAUTOCORR())),
             ("RDKitRDF", DescriptorWrapper(RDKitRDF())),
             ("RDKitMORSE", DescriptorWrapper(RDKitMORSE())),
             ("RDKitWHIM", DescriptorWrapper(RDKitWHIM())),
             # ("RDKitGETAWAY", DescriptorWrapper(RDKitGETAWAY())), # can be long
             # ("MolFeatPmapper", DescriptorWrapper(Pharmacophore3D(factory='pmapper'))), # can be long
             ("MolFeatUSRD", DescriptorWrapper(USRDescriptors())),
             ("MolFeatElectroShape", DescriptorWrapper(ElectroShapeDescriptors())),
            ]

In [23]:
network_hparams = {'hidden_layer_sizes':(256, 128, 64),
                   'num_epoch':300,
                   'batch_size':128,
                   'learning_rate':0.001,
                   'weight_decay':0.001,
                   'instance_weight_dropout':0.01,
                   'init_cuda':False,
                   'verbose':False}

In [24]:
import time
start = time.time()

w_list = [pd.DataFrame() for _ in confs_test]
for desc_name, desc_calc in desc_list:
    
    # calc descriptors
    x_train = desc_calc.transform(confs_train)
    x_test = desc_calc.transform(confs_test)

    # scale descriptors
    scaler = BagMinMaxScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # train model
    model = AttentionNetworkClassifier(**network_hparams)
    model.fit(x_train_scaled, y_train)

    # get instance weights
    w_pred = model.get_instance_weights(x_test_scaled)
    for w, df in zip(w_pred, w_list):
        df[desc_name] = w
        df.index = [f"Conformer_{i + 1}" for i in range(len(w))]

In [25]:
w_list[0].round(2) # molecule 0

,RDKitGEOM,RDKitAUTOCORR,RDKitRDF,RDKitMORSE,RDKitWHIM,MolFeatUSRD,MolFeatElectroShape
Conformer_1,0.12,0.1,0.07,0.09,0.06,0.11,0.07
Conformer_2,0.09,0.1,0.04,0.10,0.13,0.03,0.08
Conformer_3,0.10,0.1,0.10,0.10,0.11,0.64,0.09
Conformer_4,0.09,0.1,0.08,0.09,0.08,0.01,0.11
Conformer_5,0.09,0.1,0.16,0.11,0.07,0.06,0.10
Conformer_6,0.10,0.1,0.12,0.11,0.12,0.03,0.12
Conformer_7,0.11,0.1,0.08,0.09,0.10,0.02,0.11
Conformer_8,0.10,0.1,0.15,0.11,0.14,0.03,0.10
Conformer_9,0.10,0.1,0.10,0.10,0.11,0.03,0.11
Conformer_10,0.09,0.1,0.11,0.09,0.08,0.04,0.10


In [26]:
w_list[1].round(2) # molecule 1

,RDKitGEOM,RDKitAUTOCORR,RDKitRDF,RDKitMORSE,RDKitWHIM,MolFeatUSRD,MolFeatElectroShape
Conformer_1,0.11,0.1,0.05,0.09,0.06,0.02,0.12
Conformer_2,0.10,0.1,0.11,0.09,0.10,0.01,0.10
Conformer_3,0.09,0.1,0.09,0.08,0.13,0.01,0.11
Conformer_4,0.08,0.1,0.09,0.10,0.11,0.01,0.09
Conformer_5,0.10,0.1,0.06,0.10,0.14,0.04,0.11
Conformer_6,0.11,0.1,0.14,0.11,0.13,0.24,0.09
Conformer_7,0.11,0.1,0.14,0.10,0.06,0.13,0.11
Conformer_8,0.10,0.1,0.17,0.10,0.11,0.05,0.11
Conformer_9,0.10,0.1,0.06,0.11,0.10,0.08,0.07
Conformer_10,0.10,0.1,0.07,0.11,0.05,0.41,0.10


### 6. Intra-bag vs. Inter-bag

In [27]:
import numpy as np

def compute_bag_variances(bags):
    """
    Computes intra-bag and inter-bag variance from a list of bags.

    Parameters:
    - bags: list of np.ndarray, each of shape (n_instances, descriptor_dim)

    Returns:
    - intra_bag_variances: list of float (one per bag)
    - mean_intra_bag_variance: float
    - inter_bag_variance: float
    """
    bag_means = []
    intra_bag_variances = []

    for bag in bags:
        if bag.shape[0] == 0:
            raise ValueError("A bag is empty.")
        bag_mean = bag.mean(axis=0)
        bag_means.append(bag_mean)
        variance = np.mean(np.linalg.norm(bag - bag_mean, axis=1) ** 2)
        intra_bag_variances.append(variance)

    # Convert list of means to array
    bag_means = np.stack(bag_means, axis=0)
    global_mean = bag_means.mean(axis=0)

    # Inter-bag variance: variance of bag means from global mean
    inter_bag_variance = np.mean(np.linalg.norm(bag_means - global_mean, axis=1) ** 2)

    return intra_bag_variances, np.mean(intra_bag_variances), inter_bag_variance

def normalized_entropy(weights, epsilon=1e-12):
    """
    Computes normalized entropy of a vector of attention weights.

    Parameters:
    - weights: array-like of shape (n,) — non-negative, need not be normalized
    - epsilon: small value to avoid log(0)

    Returns:
    - norm_entropy: float in [0, 1], where 0 = sharp, 1 = flat
    """
    weights = np.asarray(weights, dtype=np.float64)
    weights = weights / (weights.sum() + epsilon)  # normalize

    entropy = -np.sum(weights * np.log(weights + epsilon))
    max_entropy = np.log(len(weights) + epsilon)

    return entropy / max_entropy

In [28]:
var_df = pd.DataFrame()
for desc_name, desc_calc in desc_list:

    # calc descriptors
    x_train = desc_calc.transform(confs_train)
    x_test = desc_calc.transform(confs_test)
    
    # scale bags
    bags = x_train + x_test
    scaler = BagMinMaxScaler()
    scaler.fit(bags)
    bags_scaled = scaler.transform(bags)
    
    # calc var
    intra_vars, mean_intra, mean_inter = compute_bag_variances(bags_scaled)
    
    # save results
    var_df.loc[desc_name, "intra"] = mean_intra.item()
    var_df.loc[desc_name, "inter"] = mean_inter.item()
    var_df.loc[desc_name, "ratio"] = (mean_intra / mean_inter).item()

In [29]:
var_df

,intra,inter,ratio
RDKitGEOM,0.033008,0.129730,0.254437
RDKitAUTOCORR,0.010315,1.050797,0.009817
RDKitRDF,0.054495,0.897122,0.060745
RDKitMORSE,0.174592,1.251054,0.139556
RDKitWHIM,0.337977,1.159581,0.291465
MolFeatUSRD,0.106884,0.250085,0.427389
MolFeatElectroShape,0.037707,0.207726,0.181524


In [30]:
from collections import defaultdict

ent_dict = defaultdict(list)
for bag in w_list:
    for dsc in bag.columns:
        ent_dict[dsc].append(normalized_entropy(bag[dsc]))
#
ent_df = pd.DataFrame()
for k, v in ent_dict.items():
    ent_df.loc[k, "ent"] = np.mean(v)

In [31]:
pd.concat([var_df, ent_df], axis=1).sort_values(by="ratio")

,intra,inter,ratio,ent
RDKitAUTOCORR,0.010315,1.050797,0.009817,0.999993
RDKitRDF,0.054495,0.897122,0.060745,0.950666
RDKitMORSE,0.174592,1.251054,0.139556,0.998999
MolFeatElectroShape,0.037707,0.207726,0.181524,0.991899
RDKitGEOM,0.033008,0.129730,0.254437,0.990645
RDKitWHIM,0.337977,1.159581,0.291465,0.987232
MolFeatUSRD,0.106884,0.250085,0.427389,0.716653
